In [2]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import numpy as np
import scipy as sp
import func
import phantom
import KTCFwd
import glob

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split

import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

cpu


In [3]:
class FCN(nn.Module):
    ##Neural Network
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
        'activation function'
        self.activation = nn.Tanh()
        'loss function'
        self.loss_function = nn.CrossEntropyLoss() # Change to the real loss function that we are given
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.iter = 0
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
    'foward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        a = x.float()
        for i in range(len(layers)-2):  
            z = self.linears[i](a)              
            a = self.activation(z)    
        a = self.linears[-1](a)
        return a
    
    def loss_BC(self,x,y):
                
        loss_u = self.loss_function(self.forward(x), y)
                
        return loss_u
    
    'callable for optimizer'                                       
    def closure(self):
        
        optimizer.zero_grad()
        
        loss = self.loss(X_train_Nu, U_train_Nu, X_train_Nf)
        
        loss.backward()
                
        self.iter += 1
        
        if self.iter % 100 == 0:

            error_vec, _ = PINN.test()
        
            print(loss,error_vec)

        return loss  
    
    'test neural network'
    def test(self):
                
        u_pred = self.forward(X_test)
        
        error_vec = torch.linalg.norm((u-u_pred),2)/torch.linalg.norm(u,2)        # Relative L2 Norm of the error (Vector)
        
        u_pred = u_pred.cpu().detach().numpy()
        
        u_pred = np.reshape(u_pred,(256,100),order='F')
                
        return error_vec, u_pred
    
    

In [8]:
input_folder = 'TrainingData'
groundtruth_folder = 'GroundTruths'

data_number = 1
mat_dict = sp.io.loadmat(input_folder + '/ref.mat') #load the reference data
mat_files = glob.glob(input_folder + '/data*.mat') # load the real data
reference = mat_dict['Uelref']
mat_dict2 = sp.io.loadmat(mat_files[data_number-1])
Uel = mat_dict2["Uel"]

x = torch.from_numpy(Uel.squeeze())

print(x.shape) # 1 data point

input_size = len(x)
layers = np.array([input_size,256,256*256])
PINN = FCN(layers)
print(PINN)

torch.Size([2356])
FCN(
  (activation): Tanh()
  (loss_function): CrossEntropyLoss()
  (linears): ModuleList(
    (0): Linear(in_features=2356, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=65536, bias=True)
  )
)


In [9]:
PINN.forward(x)


tensor([-0.0219,  0.0049,  0.0181,  ..., -0.0660, -0.0075,  0.0069],
       grad_fn=<ViewBackward0>)